In [2]:
import numpy as np
import pandas as pd

In [62]:
MC_compare_models_grid_test_numba = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_numba.csv', index_col=0)
tmp = MC_compare_models_grid_test_numba.drop(['heston_params#','batch_size','absolute error'], axis=1)
speed_test = pd.DataFrame()


speed_test[['strike', 'N_T']] = MC_compare_models_grid_test_numba.loc[MC_compare_models_grid_test_numba['scheme'] =='simulate_heston_euler',['strike', 'N_T']]

speed_test['qe_time'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_qe', 'time'].reset_index(drop=True)
speed_test['qe_relative_speed'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_qe', 'time'].reset_index(drop=True)/tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'time'].reset_index(drop=True)
speed_test['qe_error'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_qe', 'MC_price'].reset_index(drop=True) - tmp.loc[tmp['scheme'] =='simulate_heston_andersen_qe', 'true'].reset_index(drop=True)

speed_test['tg_time'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_tg', 'time'].reset_index(drop=True)
speed_test['tg_relative_speed'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_tg', 'time'].reset_index(drop=True)/tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'time'].reset_index(drop=True)
speed_test['tg_error'] = tmp.loc[tmp['scheme'] =='simulate_heston_andersen_tg', 'MC_price'].reset_index(drop=True) - tmp.loc[tmp['scheme'] =='simulate_heston_andersen_tg', 'true'].reset_index(drop=True)

speed_test['euler_time'] = tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'time'].reset_index(drop=True)
speed_test['euler_error'] = tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'MC_price'].reset_index(drop=True) - tmp.loc[tmp['scheme'] =='simulate_heston_euler', 'true'].reset_index(drop=True)

In [65]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'surface'}, {'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'])

tmp_euler_error = speed_test.pivot(index='strike', columns='N_T', values='euler_error').abs()
fig.add_trace(go.Surface(z=tmp_euler_error.values, x=tmp_euler_error.columns, y=tmp_euler_error.index, showscale=False), row=1, col=1, )

tmp_tg_error = speed_test.pivot(index='strike', columns='N_T', values='tg_error').abs()
fig.add_trace(go.Surface(z=tmp_tg_error.values, x=tmp_tg_error.columns, y=tmp_tg_error.index, showscale=False), row=1, col=2)

tmp_qe_error = speed_test.pivot(index='strike', columns='N_T', values='qe_error').abs()
fig.add_trace(go.Surface(z=tmp_qe_error.values, x=tmp_qe_error.columns, y=tmp_qe_error.index, showscale=False), row=1, col=3)




max_error = max(tmp_euler_error.values.max(), tmp_qe_error.values.max(), tmp_tg_error.values.max())
min_error  =min(tmp_euler_error.values.min(), tmp_qe_error.values.min(), tmp_tg_error.values.min())

fig.update_scenes(xaxis_title_text='N_T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='error',
                     zaxis = dict(nticks=4, range=[min_error*(1.02),max_error*(1.02)]))

fig.update_layout(title='Errors',
    height=700)
fig.show()

In [66]:
fig.write_image("Data/images/err_surface_strike_N_T.pdf",width = 1800, height = 700)

In [67]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'])

tmp_tg_realtive_time = speed_test.pivot(index='strike', columns='N_T', values='tg_relative_speed')
fig.add_trace(go.Surface(z=tmp_tg_realtive_time.values, x=tmp_tg_realtive_time.columns, y=tmp_tg_realtive_time.index, showscale=False), row=1, col=1)

tmp_tg_realtive_time = speed_test.pivot(index='strike', columns='N_T', values='qe_relative_speed')
fig.add_trace(go.Surface(z=tmp_tg_realtive_time.values, x=tmp_tg_realtive_time.columns, y=tmp_tg_realtive_time.index, showscale=False), row=1, col=2)




max_z = max(tmp_tg_realtive_time.values.max(), tmp_tg_realtive_time.values.max())
min_z  =min(tmp_tg_realtive_time.values.min(), tmp_tg_realtive_time.values.min())

fig.update_scenes(xaxis_title_text='N_T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='relative_speed',
                     zaxis = dict(nticks=4, range=[min_z*(1.02),max_z*(1.02)]))

fig.update_layout(title='Relative_speed',
    height=700)

fig.show()

In [69]:
fig.write_image("Data/images/relative_speed_strike_N_T.pdf",width = 1200, height = 700)

In [32]:
tmp_qe_time = speed_test.pivot(index='strike', columns='N_T', values='qe_time')
tmp_tg_time = speed_test.pivot(index='strike', columns='N_T', values='tg_time')
tmp_euler_time = speed_test.pivot(index='strike', columns='N_T', values='euler_time')

In [39]:
MC_compare_models_grid_test_asians_with_control_variate = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_asians_with_control_variate.csv', index_col=0)
MC_compare_models_grid_test_asians_without_control_variate = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_asians_without_control_variate.csv', index_col=0)

In [40]:
MC_compare_models_grid_test_asians_without_control_variate['scheme'].replace('simulate_heston_euler', 'simulate_heston_euler_no_cv', inplace = True)
MC_compare_models_grid_test_asians_without_control_variate['scheme'].replace('simulate_heston_andersen_tg', 'simulate_heston_andersen_tg_no_cv', inplace = True)
MC_compare_models_grid_test_asians_without_control_variate['scheme'].replace('simulate_heston_andersen_qe', 'simulate_heston_andersen_qe_no_cv', inplace = True)

In [45]:
test_cv = pd.concat([MC_compare_models_grid_test_asians_with_control_variate, MC_compare_models_grid_test_asians_without_control_variate])

In [44]:
test_cv

,scheme,heston_params#,strike,T,N_T,batch_size,absolute error,MC_price,time
0,simulate_heston_euler,3,50.0,1.0,10,10000,0.05,49.977559,0.320275
1,simulate_heston_euler,4,50.0,1.0,10,10000,0.05,50.011439,0.015201
2,simulate_heston_euler,5,50.0,1.0,10,10000,0.05,50.017955,0.027058
3,simulate_heston_euler,3,50.0,1.0,20,10000,0.05,49.966442,0.018862
4,simulate_heston_euler,4,50.0,1.0,20,10000,0.05,50.003481,0.029344
...,...,...,...,...,...,...,...,...,...
2515,simulate_heston_andersen_qe_no_cv,4,150.0,1.0,130,10000,0.05,0.011924,0.038158
2516,simulate_heston_andersen_qe_no_cv,5,150.0,1.0,130,10000,0.05,0.106615,0.037285
2517,simulate_heston_andersen_qe_no_cv,3,150.0,1.0,140,10000,0.05,0.000000,0.039488
2518,simulate_heston_andersen_qe_no_cv,4,150.0,1.0,140,10000,0.05,0.010454,0.039030


In [46]:
tmp_cv = test_cv.drop(['batch_size','absolute error'], axis=1)
speed_test_cv = pd.DataFrame()
speed_test_cv[['strike', 'N_T', 'heston_params#']] = test_cv.loc[test_cv['scheme'] =='simulate_heston_euler',['strike', 'N_T', 'heston_params#']]

speed_test_cv['qe_cv_speedup'] = tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_andersen_qe_no_cv', 'time'].reset_index(drop=True)/tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_andersen_qe', 'time'].reset_index(drop=True)

speed_test_cv['tg_cv_speedup'] = tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_andersen_tg_no_cv', 'time'].reset_index(drop=True)/tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_andersen_tg', 'time'].reset_index(drop=True)

speed_test_cv['euler_cv_speedup'] = tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_euler_no_cv', 'time'].reset_index(drop=True)/tmp_cv.loc[tmp_cv['scheme'] =='simulate_heston_euler', 'time'].reset_index(drop=True)


In [47]:
speed_test_cv

,strike,N_T,heston_params#,qe_cv_speedup,tg_cv_speedup,euler_cv_speedup
0,50.0,10,3,1.036954,1.061473,0.216748
1,50.0,10,4,3.980995,3.609496,4.446909
2,50.0,10,5,4.006464,4.057007,5.351786
3,50.0,20,3,3.944814,3.953057,5.234811
4,50.0,20,4,3.456627,3.615132,3.719907
...,...,...,...,...,...,...
835,150.0,130,4,0.856814,0.817845,0.893644
836,150.0,130,5,0.854268,0.993839,0.897874
837,150.0,140,3,0.863650,0.836486,0.890616
838,150.0,140,4,0.837527,0.829133,0.891230


In [148]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'surface'},{'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'])

paramns_n = 5


euler_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='euler_cv_speedup')
fig.add_trace(go.Surface(z=euler_cv_speedup.values, x=euler_cv_speedup.columns, y=euler_cv_speedup.index, showscale=False), row=1, col=1)

tg_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='tg_cv_speedup')
fig.add_trace(go.Surface(z=tg_cv_speedup.values, x=tg_cv_speedup.columns, y=tg_cv_speedup.index, showscale=False), row=1, col=2)

qe_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='qe_cv_speedup')
fig.add_trace(go.Surface(z=qe_cv_speedup.values, x=qe_cv_speedup.columns, y=qe_cv_speedup.index, showscale=False), row=1, col=3)

max_z = max(euler_cv_speedup.max().max(),tg_cv_speedup.values.max().max(), qe_cv_speedup.max().max())
min_z  =min(euler_cv_speedup.min().min(), tg_cv_speedup.values.min().min(), qe_cv_speedup.values.min().min())


fig.update_scenes(xaxis_title_text='N_T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='Speed up',
                     zaxis = dict(nticks=4, range=[min_z*(1.02),max_z*(1.02)]))

fig.update_layout(#title='Control Variate Speed up Parameters #' + str(paramns_n),
    height=700)

fig.show()

In [150]:
fig.write_image("surf_" + str(paramns_n) + ".pdf",width = 1800, height = 700)

In [146]:

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'], shared_yaxes=True)

paramns_n = 5

srez  = 80

euler_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='euler_cv_speedup')[srez]
fig.add_trace(go.Scatter(x= euler_cv_speedup.index , y= euler_cv_speedup.values), row=1, col=1,)

tg_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='tg_cv_speedup')[srez]
fig.add_trace(go.Scatter(x= tg_cv_speedup.index , y= tg_cv_speedup.values), row=1, col=2)

qe_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='qe_cv_speedup')[srez]
fig.add_trace(go.Scatter(x= qe_cv_speedup.index , y= qe_cv_speedup.values), row=1, col=3)

fig.update_layout(title='Control Variate Speed up Parameters #' + str(paramns_n)  + ' at N_T=' + str(srez),
    height=700,
    showlegend=False)


fig.update_yaxes(title_text="Speed up", row=1, col=1)
fig.update_yaxes(title_text="Speed up", row=1, col=2)
fig.update_yaxes(title_text="Speed up", row=1, col=3)
fig.update_xaxes(title_text="Strike", row=1, col=1)
fig.update_xaxes(title_text="Strike", row=1, col=2)
fig.update_xaxes(title_text="Strike", row=1, col=3)

fig.show()

In [147]:
fig.write_image("pot_" + str(paramns_n) + ".pdf",width = 1800, height = 700)

In [73]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=3, specs=[[{'type':'surface'},{'type':'surface'}, {'type':'surface'}], [{'type':'surface'},{'type':'surface'}, {'type':'surface'}], [{'type':'surface'},{'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'])

i = 1

max_z = -10000
min_z = 10000

for paramns_n in speed_test_cv['heston_params#'].unique():
    euler_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='euler_cv_speedup').abs()
    fig.add_trace(go.Surface(z=euler_cv_speedup.values, x=euler_cv_speedup.columns, y=euler_cv_speedup.index, showscale=False), row=i, col=1)

    tg_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='tg_cv_speedup').abs()
    fig.add_trace(go.Surface(z=tg_cv_speedup.values, x=tg_cv_speedup.columns, y=tg_cv_speedup.index, showscale=False), row=i, col=2)

    qe_cv_speedup = speed_test_cv[speed_test_cv['heston_params#'] ==paramns_n ].pivot(index='strike', columns='N_T', values='qe_cv_speedup').abs()
    fig.add_trace(go.Surface(z=qe_cv_speedup.values, x=qe_cv_speedup.columns, y=qe_cv_speedup.index, showscale=False), row=i, col=3)
    i += 1

    max_z = max(max_z, euler_cv_speedup.max().max(),tg_cv_speedup.values.max().max(), qe_cv_speedup.max().max())
    min_z  =min(max_z, euler_cv_speedup.min().min(), tg_cv_speedup.values.min().min(), qe_cv_speedup.values.min().min())


fig.update_scenes(xaxis_title_text='N_T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='relative_speed',
                     zaxis = dict(nticks=4, range=[min_z*(1.02),max_z*(1.02)]))

fig.update_layout(title='Control Variate Speed up',
    height=2100);

#fig.show()

In [78]:
MC_compare_models_grid_test_3_andersen  = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_3_andersen.csv', index_col=0)
MC_compare_models_grid_test_3_euler = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_3_euler.csv', index_col=0)
MC_compare_models_grid_test_3_qe = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_3_qe.csv', index_col=0)
MC_compare_models_grid_test_3_qe = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_3_PART3.csv', index_col=0)
test = pd.concat([MC_compare_models_grid_test_3_andersen, MC_compare_models_grid_test_3_euler, MC_compare_models_grid_test_3_qe])

In [83]:
scheme_  = 'simulate_heston_euler_cupy'
N_T_ = 175
heston_params_n_ = 5

df = test

test2_euler = df.loc[(df['scheme'] == scheme_) &(df['N_T'] == N_T_) &(df['heston_params#'] == heston_params_n_)]

test2_euler = test2_euler[['strike', 'T', 'MC_price', 'true']]




scheme_  = 'simulate_heston_andersen_tg_cupy'

test2_anderson_qe = df.loc[(df['scheme'] == scheme_) &(df['N_T'] == N_T_) &(df['heston_params#'] == heston_params_n_)]

test2_anderson_qe = test2_anderson_qe[['strike', 'T', 'MC_price', 'true']]




scheme_  = 'simulate_heston_andersen_tg_cupy'

test2_anderson_tg = df.loc[(df['scheme'] == scheme_) &(df['N_T'] == N_T_) &(df['heston_params#'] == heston_params_n_)]


test2_anderson_tg = test2_anderson_tg[['strike', 'T', 'MC_price', 'true']]


test2_euler['error'] = (test2_euler['MC_price'] - test2_euler['true'])#/test2_euler['true']
test2_anderson_tg['error'] = (test2_anderson_tg['MC_price'] - test2_anderson_tg['true'])#/test2_anderson_tg['true']
test2_anderson_qe['error'] = (test2_anderson_qe['MC_price'] - test2_anderson_qe['true'])#/test2_anderson_qe['true']


test2_euler = test2_euler.pivot(index='strike', columns='T', values='error')
test2_anderson_tg = test2_anderson_tg.pivot(index='strike', columns='T', values='error')
test2_anderson_qe = test2_anderson_qe.pivot(index='strike', columns='T', values='error')


In [84]:
test2_euler = test2_euler.abs()
test2_anderson_tg = test2_anderson_tg.abs()
test2_anderson_qe = test2_anderson_qe.abs()

In [86]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'surface'}, {'type':'surface'}, {'type':'surface'}]], subplot_titles = ['Euler', 'Andersen Truncated Gaussian', 'Andersen Quadratic-Exponential'], shared_xaxes=True)


max_error = max(test2_euler.values.max(), test2_anderson_tg.values.max(), test2_anderson_qe.values.max())
min_error  =min(test2_euler.values.min(), test2_anderson_tg.values.min(), test2_anderson_qe.values.min())

fig.add_trace(go.Surface(z=test2_euler.values, x=test2_euler.columns, y=test2_euler.index, showscale=False), row=1, col=1, )
fig.add_trace(go.Surface(z=test2_anderson_tg.values, x=test2_anderson_tg.columns, y=test2_anderson_tg.index, showscale=False), row=1, col=2)
fig.add_trace(go.Surface(z=test2_anderson_qe.values, x=test2_anderson_qe.columns, y=test2_anderson_qe.index, showscale=False), row=1, col=3)
fig.update_scenes(xaxis_title_text='T',  
                  yaxis_title_text='Strike',  
                  zaxis_title_text='error',
                     zaxis = dict(nticks=4, range=[min_error*(1.02),max_error*(1.02)]))

fig.update_layout(
    height=700);
fig.show()

In [56]:
fig.write_image("Data/images/err_surface_strike_T_N_T=" + str(N_T_) +"_param#" + str(heston_params_n_) + ".pdf",width = 1800, height = 700)